In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

/mnt/data4/haryoaw_workspace/projects/2021_2/s4_happy/happy_s4


In [2]:
from happy_s4.data.lit_data import HFLitDataArgs, HFLitDataModule

/mnt/data3/shared_envs/s4/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from happy_s4.model.lit_model import LitS4Args, LitS4
from happy_s4.model.s4_wrap import S4_GO_BRR_ARGS

CUDA extension for cauchy multiplication not found. Install by going to extensions/cauchy/ and running `python setup.py install`. This should speed up end-to-end training by 10-50%


In [4]:
lit_args = HFLitDataArgs(min_freq=1, split_pattern=r"")

In [5]:
dm = HFLitDataModule(lit_args)

In [6]:
dm.setup()

Reusing dataset glue (/mnt/data1/hf_dataset_cache/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Loading cached processed dataset at /mnt/data1/hf_dataset_cache/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-4c94838283d0c555.arrow
Reusing dataset glue (/mnt/data1/hf_dataset_cache/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Loading cached processed dataset at /mnt/data1/hf_dataset_cache/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-6f8a7dfd1c2edfb8.arrow


In [7]:
vocab_size = len(dm.tokenizer.vocab.get_itos())
pad_token_id = dm.tokenizer.pad_token_id


In [8]:
vocab_size

84

In [9]:
s4_args = S4_GO_BRR_ARGS(pad_token_id=pad_token_id, vocab_size=vocab_size, d_model=512, l_max=3000, channels=1,hurwitz=True, transposed=False)

In [10]:
lit_s4_args = LitS4Args(s4_args=s4_args)

In [11]:
lit_s4 = LitS4(lit_s4_args)

In [12]:
dl = dm.val_dataloader()

In [13]:
from pytorch_lightning import Trainer

In [14]:
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

In [15]:
es = EarlyStopping(monitor="val_loss")
mckpt = ModelCheckpoint(dirpath="outputs/mrpc-char/", filename="checkpoint_{epoch:02d}-{val_loss:03.0f}", monitor='val_loss', )

In [16]:
trainer = Trainer(callbacks=[es, mckpt], gpus=[1], max_epochs=1000)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [17]:
trainer.fit(model=lit_s4, datamodule=dm)

Reusing dataset glue (/mnt/data1/hf_dataset_cache/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 679.72it/s]
Reusing dataset glue (/mnt/data1/hf_dataset_cache/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Loading cached processed dataset at /mnt/data1/hf_dataset_cache/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-4c94838283d0c555.arrow
Reusing dataset glue (/mnt/data1/hf_dataset_cache/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Loading cached processed dataset at /mnt/data1/hf_dataset_cache/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-6f8a7dfd1c2edfb8.arrow
/mnt/data3/shared_envs/s4/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:611: UserWarning: Checkpoint dire

Sanity Checking DataLoader 0:   0%|                                                               | 0/2 [00:00<?, ?it/s]

/mnt/data3/shared_envs/s4/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/mnt/data3/shared_envs/s4/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0:  90%|█████████████████████████████████████████████▊     | 115/128 [00:03<00:00, 30.21it/s, loss=0.684, v_num=4]
Validation: 0it [00:00, ?it/s]
Epoch 1:  90%|███████████████████████████████████████████████▌     | 115/128 [00:07<00:00, 15.12it/s, loss=0.6, v_num=4]
Validation: 0it [00:00, ?it/s]
Epoch 2:  90%|█████████████████████████████████████████████▊     | 115/128 [00:11<00:01, 10.09it/s, loss=0.583, v_num=4]
Validation: 0it [00:00, ?it/s]
Epoch 3:  90%|██████████████████████████████████████████████▋     | 115/128 [00:14<00:01,  7.69it/s, loss=0.57, v_num=4]
Validation: 0it [00:00, ?it/s]
Epoch 4:  90%|█████████████████████████████████████████████▊     | 115/128 [00:18<00:02,  6.22it/s, loss=0.559, v_num=4]
Validation: 0it [00:00, ?it/s]
Epoch 5:  90%|█████████████████████████████████████████████▊     | 115/128 [00:22<00:02,  5.17it/s, loss=0.548, v_num=4]
Validation: 0it [00:00, ?it/s]
Epoch 5: 100%|███████████████████████████████████████████████████| 128/128 [00:22<00:00,

In [31]:
lit_s4 = lit_s4.load_from_checkpoint("outputs/mrpc-char/checkpoint_epoch=02-val_loss=001.ckpt")

In [32]:
from happy_s4.data.data import WordTokenizer, shape_dataset
from happy_s4.data.collators import BatchCollators

In [33]:
import datasets
from functools import partial

In [34]:
from torch.utils.data import DataLoader

In [35]:
data_test = datasets.load_dataset(
    "glue",
    "mrpc",
    split="test",
)
data_test = data_test.map(
    partial(shape_dataset, word_tokenizer=dm.tokenizer)
)


Reusing dataset glue (/mnt/data1/hf_dataset_cache/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Loading cached processed dataset at /mnt/data1/hf_dataset_cache/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-5bc522fcd8cb7341.arrow


In [36]:
data_test = data_test.remove_columns(['sentence1', 'sentence2', 'idx'])

In [37]:
dl = DataLoader(
    dataset=data_test,
    collate_fn=BatchCollators(dm.tokenizer.pad_token_id),
    batch_size=dm.args.batch_size,
    shuffle=False
)

In [38]:
import torch

In [39]:
preds = []
gt = []

In [40]:
from tqdm import tqdm

In [44]:
lit_s4.eval()

LitS4(
  (model): S4_GO_BRR_Classification(
    (backbone): S4_GO_BRR(
      (s4): S4(
        (kernel): HippoSSKernel(
          (kernel): SSKernelNPLR()
        )
        (activation): GELU()
        (dropout): Identity()
        (output_linear): Linear(in_features=512, out_features=512, bias=True)
      )
      (embedding): Embedding(84, 512, padding_idx=0)
    )
  )
)

tensor([ 6, 68, 14, 24, 16, 69,  6,  6, 21, 19, 17, 13,  7, 18, 18, 12, 21, 12,
        10, 26, 13, 10, 27, 13, 21, 19, 17, 13, 19, 10, 15, 16, 17, 13, 16, 19,
        10, 10, 21, 12, 26, 29, 13, 17, 48, 32,  7, 26, 18, 16, 13, 21, 19, 17,
        13, 12, 26, 30, 17, 16, 21, 12, 29,  7, 21, 12, 10, 26, 13,  7,  9, 17,
         7, 13, 17,  7, 16, 21, 13, 20, 28, 13, 21, 23, 10, 13,  8, 12, 24, 17,
        16, 13, 22, 13, 23, 12, 21, 19, 13, 21, 19, 17, 13, 32, 10, 24, 12, 14,
        17, 13, 26, 10, 23, 13, 17, 48,  7,  8, 12, 26, 12, 26, 29, 13,  7, 13,
        16, 17, 30, 17, 26, 50,  8, 12, 24, 17, 13, 16, 17, 14, 21, 12, 10, 26,
        13, 10, 27, 13, 21, 19, 17, 13, 27,  9, 17, 17, 23,  7, 28, 13, 31,  6,
         6, 68, 16, 17, 32, 69,  6,  6, 21, 19, 17, 13, 19, 10, 15, 16, 17, 13,
        16, 19, 10, 10, 21, 12, 26, 29, 13, 17, 48, 32,  7, 26, 18, 16, 13, 21,
        19, 17, 13, 12, 26, 30, 17, 16, 21, 12, 29,  7, 21, 12, 10, 26, 13,  7,
         9, 17,  7, 13, 17,  7, 16, 21, 

In [59]:
[dm.tokenizer.vocab.get_itos()[x] for x in batch['input_ids'][16]]

['',
 '[',
 'c',
 'l',
 's',
 ']',
 '',
 '',
 't',
 'h',
 'e',
 ' ',
 'a',
 'd',
 'd',
 'i',
 't',
 'i',
 'o',
 'n',
 ' ',
 'o',
 'f',
 ' ',
 't',
 'h',
 'e',
 ' ',
 'h',
 'o',
 'u',
 's',
 'e',
 ' ',
 's',
 'h',
 'o',
 'o',
 't',
 'i',
 'n',
 'g',
 ' ',
 'e',
 'x',
 'p',
 'a',
 'n',
 'd',
 's',
 ' ',
 't',
 'h',
 'e',
 ' ',
 'i',
 'n',
 'v',
 'e',
 's',
 't',
 'i',
 'g',
 'a',
 't',
 'i',
 'o',
 'n',
 ' ',
 'a',
 'r',
 'e',
 'a',
 ' ',
 'e',
 'a',
 's',
 't',
 ' ',
 'b',
 'y',
 ' ',
 't',
 'w',
 'o',
 ' ',
 'm',
 'i',
 'l',
 'e',
 's',
 ' ',
 ',',
 ' ',
 'w',
 'i',
 't',
 'h',
 ' ',
 't',
 'h',
 'e',
 ' ',
 'p',
 'o',
 'l',
 'i',
 'c',
 'e',
 ' ',
 'n',
 'o',
 'w',
 ' ',
 'e',
 'x',
 'a',
 'm',
 'i',
 'n',
 'i',
 'n',
 'g',
 ' ',
 'a',
 ' ',
 's',
 'e',
 'v',
 'e',
 'n',
 '-',
 'm',
 'i',
 'l',
 'e',
 ' ',
 's',
 'e',
 'c',
 't',
 'i',
 'o',
 'n',
 ' ',
 'o',
 'f',
 ' ',
 't',
 'h',
 'e',
 ' ',
 'f',
 'r',
 'e',
 'e',
 'w',
 'a',
 'y',
 ' ',
 '.',
 '',
 '',
 '[',
 's',
 'e',
 'p',
 ']

In [45]:
for batch in tqdm(dl):
    with torch.no_grad(): 
        _, logits = lit_s4(input_ids=batch['input_ids'], labels=batch['label'])
        label = logits.argmax(dim=1).numpy().tolist()
        gt.extend(batch['label'].numpy().tolist())
        preds.extend(label)

100%|███████████████████████████████████████████████████████████████████████████████████| 54/54 [00:04<00:00, 12.92it/s]


In [46]:
from sklearn.metrics import accuracy_score

In [47]:
accuracy_score(gt, preds)

0.6689855072463768